<a href="https://colab.research.google.com/github/dmscornjs/Hotelreview/blob/main/%EC%97%AC%EA%B8%B0%EC%96%B4%EB%95%8C%ED%81%AC%EB%A1%A4%EB%A7%81%EC%A0%9C%EC%B6%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#여기어때 크롤링 
* 분석 환경:주피터 노트북

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

여기어때의 경우 호텔로 분류되는 되는 것 중 지역이 서울에 해당하는 호텔들을 전부 조사하였다.  
야놀자와 달리 서울 전체로 구분되지 않고 강남, 명동과 같이 서울의 지역 단위로 구분되어져 있었다.  
그래서 서울 지역에 해당하는 호텔 URL을 모아 모두 수집하였다.

In [ ]:
seoullist=["https://www.goodchoice.kr/product/search/2/2012", #강남/역삼/삼성/신사/청담
           "https://www.goodchoice.kr/product/search/2/2019", #서초/교대
           "https://www.goodchoice.kr/product/search/2/2016", #잠실
           "https://www.goodchoice.kr/product/search/2/2014", #을지로
           "https://www.goodchoice.kr/product/search/2/2015", #종로
           "https://www.goodchoice.kr/product/search/2/2020", #서울역
           "https://www.goodchoice.kr/product/search/2/2018", #마포
           "https://www.goodchoice.kr/product/search/2/2017", #영등포
           "https://www.goodchoice.kr/product/search/2/2021" #구로
]

In [ ]:
# url 함수
def make_url(Url):
  #driver = webdriver.Chrome('chromedriver')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(Url)

    html = driver.page_source
    soup = bs(html, 'html.parser')
    data = soup.select("div > li > a")

    for d in data:
        hotelurl.append(d.attrs['href'])
    driver.close()

# 크롤링 함수
def crawl(start, end):
    global df
    for p in range(start,end):
        html = driver.page_source
        soup = bs(html, 'html.parser')

        reviews = soup.select('#review > ul > li')
        
        for r in reviews:
            review = r.select_one('.txt').text
            star = r.select_one('.score_wrap_sm .num').text
            date = r.select_one('.date').text
            #hotels.append(hotel)
            df = df.append(pd.DataFrame([[hotel, star, review, date]], columns = columns),ignore_index=True)

        driver.find_element_by_xpath('//*[@id="pagination"]/div/button['+str(p)+']').click()
        time.sleep(1)
        #print(soup.select('.paging .on')[0].text)

def make_review_crawl(url):
    global df
    global hotel
    driver.get(url)
    # 페이지 파싱
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # '리뷰' 클릭하도록 명령
    hotel = driver.find_element_by_css_selector('h2').text
    hotel = hotel.replace("[반짝특가] ", "")
    #hotels.append(hotel)
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/button[3]').click()
    
    top_text = soup.select('.score_top')[0].find('strong').text
    top_score = soup.select('.score_top .num')[0].text
    top_total=soup.select('.score_top')[0].find('p').find('b').text

    # 리뷰 총 개수
    top_total = int(soup.select('.score_top')[0].find('p').find('b').text)

    # 마지막 페이지 수 (한페이지에 리뷰 10개씩)
    last_page = top_total // 10 + 1 

    # next_page 몇 번 클릭 해야하는지 설정
    if last_page % 5 == 0:
        page_step = last_page // 5 
    else:
        page_step = last_page // 5 + 1   
    
    print(top_total, last_page, page_step)
    step = 1
    while(page_step >= step):
        
        # 첫번째 step
        if step == 1:
            print('step1 loop')
            crawl(2,7)
            
        # 마지막 step
        elif step == page_step:
            print('step 마지막 loop')
                
            if top_total % 50 > 10:
                            
                if last_page % 5 == 0:
                    end_page = 7
                else: 
                    end_page =last_page % 5 + 2
                    
                crawl(3, end_page)
                
            # 마지막 페이지
            html = driver.page_source
            soup = bs(html, 'html.parser')
            reviews = soup.select('#review > ul > li')
        
            for r in reviews:
                review = r.select_one('.txt').text
                star = r.select_one('.score_wrap_sm .num').text
                date = select_one('.date').text
                hotels.append(hotel)
                df = df.append(pd.DataFrame([[hotel, star, review, date]], columns = columns),ignore_index=True)
                print(date, hotel)   
        # 중간 step
        else:
            crawl(3,8)
        step += 1

    df.append(pd.DataFrame({'review': review, 'star': star,'date':date }))
    df['hotel'] = hotel

columns = ['hotel', 'star', 'review','date']
df = pd.DataFrame(columns = columns) # 데이터프레임 생성


In [ ]:
hotelurl = []
for Url in seoullist:
  make_url(Url)

In [ ]:
hotelurl

['https://www.goodchoice.kr/product/detail?ano=7936&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=12877&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=6903&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=57843&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=6264&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=6826&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=6533&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=65455&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=6490&adcno=2&sel_date=2022-04-14&sel_date2=2022-04-15',
 'https://www.goodchoice.kr/product/detail?ano=69906&adcno=2&sel_date=

In [ ]:
dfhotelurl = pd.DataFrame({'url':hotelurl})
dfhotelurl

,url
0,https://www.goodchoice.kr/product/detail?ano=7...
1,https://www.goodchoice.kr/product/detail?ano=1...
2,https://www.goodchoice.kr/product/detail?ano=6...
3,https://www.goodchoice.kr/product/detail?ano=5...
4,https://www.goodchoice.kr/product/detail?ano=6...
...,...
303,https://www.goodchoice.kr/product/detail?ano=6...
304,https://www.goodchoice.kr/product/detail?ano=9...
305,https://www.goodchoice.kr/product/detail?ano=6...
306,https://www.goodchoice.kr/product/detail?ano=4...


In [ ]:
dfhotelurl.describe()

,url
count,308
unique,252
top,https://www.goodchoice.kr/product/detail?ano=6...
freq,2


In [ ]:
dfhotelurl.to_csv("hotelurl0414_270.csv")

마지막 리뷰 페이지가 빈 페이지이거나 수집 중 정보가 변경되는 등으로 인해 오류가 발생하였다.  
이 경우 예외처리를 통해 끊기지 않고 수집할 수 있도록 하였다.

In [ ]:
review = []
star = []
hotels = []
date = []
ex = []

for u in hotelurl:
  # 페이지 로딩 3초 대기
    driver = webdriver.Chrome(ChromeDriverManager().install())
    try:
        make_review_crawl(u)
    except:
        ex.append(u)
    driver.close()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\ynk99\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


895 90 18
step1 loop
step 마지막 loop




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\ynk99\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


991 100 20
step1 loop
step 마지막 loop




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\ynk99\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


...

11 2 1
step1 loop




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\ynk99\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


589 59 12
step1 loop
step 마지막 loop




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\ynk99\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


260 27 6
step1 loop
step 마지막 loop


In [ ]:
df

,hotel,star,review,date
0,신라스테이 역삼,8.0,출장으로 역삼 근처 호텔 찾아보다가 신라스테이 근처에 센터필드도 있고 밥집 카페도 ...,20시간 전
1,신라스테이 역삼,10.0,잘쉬디ㅡ으ㅡㅁ잘쉬디ㅡ으ㅡㅁ잘쉬디ㅡ으ㅡㅁ잘쉬디ㅡ으ㅡㅁ잘쉬디ㅡ으ㅡㅁ잘쉬디ㅡ으ㅡㅁ잘쉬디ㅡ...,10일 전
2,신라스테이 역삼,10.0,깔끔하고 고급스러웠습니다 가격대비 만족했던곳이라 넘넘 만족스러웠어요 로비도 깔끔하고...,12일 전
3,신라스테이 역삼,10.0,직원분들 너무 친절하십니다 안내데스크가 넓어서 잠시 쉬기에도 편합니다 편의점이 바로...,24일 전
4,신라스테이 역삼,10.0,깔끔하고 서울에서 갑자기 일이 있거나 놀게 되었을 때 가장 합리적인 선택이라고 생각...,24일 전
...,...,...,...,...
143896,뜨랑블루 호텔,4.3,시설 깨끗하고 다 좋은데퇴실도 안햇는데 청소하시는분인지 입구문을 맘대로 여시더라구요...,5년 전
143897,뜨랑블루 호텔,10.0,깔끔하고 친철하고 모두 만족해요꼭 한번더 오고 싶은 숙박업소입니다.,5년 전
143898,뜨랑블루 호텔,10.0,괜찮았어요 좋더라구요 마음에 들었어요냄새도 안나드러구요,5년 전
143899,뜨랑블루 호텔,10.0,시설 신림권에서 최고청결하고 수건에서 냄새도 안나고흠이라면 1000원받는 일회용품이...,5년 전


In [ ]:
df.to_csv("여기어때_크롤링.csv", index = False)